In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv1D, MaxPool1D
from keras.layers import Embedding, Dense, LSTM, Activation, Dropout, Flatten
from tensorflow.keras.optimizers import Adam

Init Plugin
Init Graph Optimizer
Init Kernel


In [2]:
import pandas as pd
import numpy as np
import  seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [31]:
aus=pd.read_table('./Australian/australian.dat', header=None,sep=' ')
colnames=['A1','A2','A3','A4','A5','A6','A7','A8','A9','A10','A11','A12','A13','A14','Label']
aus.columns=colnames

In [32]:
aus_pre=aus.copy(deep=True)
aus_pre=aus_pre.drop(columns=['Label'])
pre_aus=StandardScaler().fit_transform(aus_pre)
pre_aus=pd.DataFrame(pre_aus,columns = aus_pre.select_dtypes(include='number').columns)
pre_aus=pd.concat([pre_aus, aus[['Label']]], axis=1)

In [33]:
from sklearn.ensemble import RandomForestClassifier
X, y = pre_aus.loc[:, np.delete(pre_aus.columns.values, 
                                        np.where(pre_aus.columns.values == ['Label']))], \
        pre_aus.loc[:, 'Label']

# train on randomForest to get important features
clf = RandomForestClassifier(n_estimators=100, max_depth=2,
                             random_state=0)
clf.fit(X, y) 

feature_importances = pd.DataFrame(sorted(zip(pre_aus.columns, clf.feature_importances_), key=lambda x: x[1] * -1),
                                    columns = ['feature','importance'])


In [34]:
top_features = feature_importances[feature_importances.importance>0]['feature'].values

X, y = pre_aus.loc[:,top_features], pre_aus.loc[:,'Label']

In [35]:
y=y*(-1)+1
#0 po 1 ne
#0 po -1 ne
#+1

In [36]:
from imblearn.over_sampling import SMOTE

In [37]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0,stratify = y)
sm = SMOTE(random_state=0,sampling_strategy=1)
x_train_b, y_train_b = sm.fit_resample(x_train, y_train.ravel())

In [10]:
from scipy.stats import ks_2samp

In [11]:
def ks_stat(y, yhat):
    return ks_2samp(yhat[y==1], yhat[y!=1]).statistic

In [12]:
def type2_calcu(y,yhat):
    confusion =confusion_matrix(y,yhat)
    #[row, column]
    TP = confusion[1, 1]
    TN = confusion[0, 0]
    FP = confusion[0, 1]
    FN = confusion[1, 0]
    return FP / float(FP + TN)
def type1_calcu(y,yhat):
    confusion =confusion_matrix(y,yhat)
    #[row, column]
    TP = confusion[1, 1]
    TN = confusion[0, 0]
    FP = confusion[0, 1]
    FN = confusion[1, 0]
    return FN / float(TP + FN)

In [13]:
def scores(model):
    train=[]
    test=[]
    y_train_c=model.predict(x_train)
    y_train_pre=model.predict_proba(x_train)
    y_pred_c=model.predict(x_test)
    y_pred = model.predict_proba(x_test)
    #roc
    train.append(roc_auc_score(y_train,y_train_pre))
    test.append(roc_auc_score(y_test, y_pred))
    #ks
    train.append(ks_stat(y_train,np.round(y_train_c[:,0],0)))
    test.append(ks_stat(y_test, np.round(y_pred_c[:,0],0)))
    #brier
    train.append(brier_score_loss(y_train,y_train_pre))
    test.append(brier_score_loss(y_test, y_pred))
    #acc
    train.append(accuracy_score(y_train,np.round(y_train_c,0)))
    test.append(accuracy_score(y_test, np.round(y_pred_c,0)))
    #t1
    train.append(type1_calcu(y_train,np.round(y_train_c,0)))
    test.append(type1_calcu(y_test, np.round(y_pred_c,0)))
    #t2
    train.append(type2_calcu(y_train,np.round(y_train_c,0)))
    test.append(type2_calcu(y_test, np.round(y_pred_c,0)))
    return train,test

In [14]:
print(x_train.shape)

(552, 13)


In [38]:
x_train=StandardScaler().fit_transform(x_train)
x_test=StandardScaler().fit_transform(x_test)

x_train_b=StandardScaler().fit_transform(x_train_b)

In [39]:
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], 1)

x_train_b = x_train_b.reshape(x_train_b.shape[0], x_train_b.shape[1], 1)

In [17]:
epochs = 20
model = Sequential()
model.add(Conv1D(32, 2, activation='relu',input_shape = x_train[0].shape))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv1D(64, 2, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(1, activation='sigmoid'))

Metal device set to: Apple M1 Pro


2022-07-27 20:40:46.654824: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-07-27 20:40:46.655063: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 12, 32)            96        
_________________________________________________________________
batch_normalization (BatchNo (None, 12, 32)            128       
_________________________________________________________________
module_wrapper (ModuleWrappe (None, 12, 32)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 11, 64)            4160      
_________________________________________________________________
batch_normalization_1 (Batch (None, 11, 64)            256       
_________________________________________________________________
module_wrapper_1 (ModuleWrap (None, 11, 64)            0         
_________________________________________________________________
module_wrapper_2 (ModuleWrap (None, 704)               0

In [19]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss = 'binary_crossentropy', metrics=['accuracy'])

In [20]:
history = model.fit(x_train, y_train, epochs=20, verbose=1)

2022-07-27 20:40:54.119133: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2022-07-27 20:40:54.119384: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/20


2022-07-27 20:40:54.432052: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


18/18 [==============================] - 1s 19ms/step - loss: 0.9583 - accuracy: 0.6069
Epoch 2/20
18/18 [==============================] - 0s 15ms/step - loss: 0.9252 - accuracy: 0.5851
Epoch 3/20
18/18 [==============================] - 0s 15ms/step - loss: 0.7865 - accuracy: 0.6504
Epoch 4/20
18/18 [==============================] - 0s 15ms/step - loss: 0.7020 - accuracy: 0.7029
Epoch 5/20
18/18 [==============================] - 0s 15ms/step - loss: 0.6754 - accuracy: 0.7319
Epoch 6/20
18/18 [==============================] - 0s 15ms/step - loss: 0.5574 - accuracy: 0.7554
Epoch 7/20
18/18 [==============================] - 0s 16ms/step - loss: 0.6118 - accuracy: 0.7337
Epoch 8/20
18/18 [==============================] - 0s 16ms/step - loss: 0.5936 - accuracy: 0.7464
Epoch 9/20
18/18 [==============================] - 0s 16ms/step - loss: 0.5945 - accuracy: 0.7554
Epoch 10/20
18/18 [==============================] - 0s 16ms/step - loss: 0.5283 - accuracy: 0.7717
Epoch 11/20
18/18 [=

In [21]:
from sklearn.metrics import roc_auc_score, recall_score, precision_score,make_scorer,confusion_matrix,brier_score_loss,accuracy_score

In [22]:
print(scores(model))

([0.9341224294595888, 0.6889845369041926, 0.11099421065014452, 0.8514492753623188, 0.0915032679738562, 0.21951219512195122], [0.89589099425165, 0.6700021290185224, 0.12560885983570785, 0.8405797101449275, 0.11688311688311688, 0.21311475409836064])


2022-07-27 20:41:06.280780: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
/Users/weixiao/miniforge3/envs/tensorflow/lib/python3.9/site-packages/tensorflow/python/keras/engine/sequential.py:430: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


In [40]:
model2 = Sequential()
model2.add(LSTM(32, input_shape=(1,13), activation='relu', return_sequences=True))
model2.add(Dropout(0.2))

model2.add(LSTM(64, activation='relu'))
model2.add(Dropout(0.5))

model2.add(Dense(64, activation='relu'))
model2.add(Dropout(0.5))

model2.add(Flatten())
model2.add(Dense(1, activation='sigmoid'))

In [41]:
x_train = x_train.reshape(x_train.shape[0], 1,x_train.shape[1])
x_test = x_test.reshape(x_test.shape[0], 1,x_test.shape[1])

In [42]:
x_train.shape

(552, 1, 13)

In [43]:
model2.compile(optimizer=Adam(learning_rate=0.001), loss = 'binary_crossentropy', metrics=['accuracy'])

In [44]:
history=model2.fit(x_train, y_train, epochs=20)

Epoch 1/20


2022-07-27 20:50:36.235064: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


18/18 [==============================] - 2s 25ms/step - loss: 0.6915 - accuracy: 0.5272
Epoch 2/20
18/18 [==============================] - 0s 20ms/step - loss: 0.6820 - accuracy: 0.5833
Epoch 3/20
18/18 [==============================] - 0s 20ms/step - loss: 0.6628 - accuracy: 0.6522
Epoch 4/20
18/18 [==============================] - 0s 20ms/step - loss: 0.6113 - accuracy: 0.7736
Epoch 5/20
18/18 [==============================] - 0s 21ms/step - loss: 0.5127 - accuracy: 0.8315
Epoch 6/20
18/18 [==============================] - 0s 20ms/step - loss: 0.4026 - accuracy: 0.8533
Epoch 7/20
18/18 [==============================] - 0s 22ms/step - loss: 0.3562 - accuracy: 0.8696
Epoch 8/20
18/18 [==============================] - 0s 22ms/step - loss: 0.3549 - accuracy: 0.8786
Epoch 9/20
18/18 [==============================] - 0s 22ms/step - loss: 0.3288 - accuracy: 0.8696
Epoch 10/20
18/18 [==============================] - 0s 20ms/step - loss: 0.2964 - accuracy: 0.8913
Epoch 11/20
18/18 [=

In [45]:
print(scores(model2))

2022-07-27 20:50:47.711961: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
/Users/weixiao/miniforge3/envs/tensorflow/lib/python3.9/site-packages/tensorflow/python/keras/engine/sequential.py:430: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


([0.9607843137254902, 0.787342579308146, 0.07557684769537518, 0.894927536231884, 0.09477124183006536, 0.11788617886178862], [0.9222908239301683, 0.7355758995103258, 0.10252699131296286, 0.8695652173913043, 0.11688311688311688, 0.14754098360655737])


In [26]:
epochs = 20
model3 = Sequential()
model3.add(Conv1D(32, 2, activation='relu',input_shape = x_train_b[0].shape))
model3.add(BatchNormalization())
model3.add(Dropout(0.2))

model3.add(Conv1D(64, 2, activation='relu'))
model3.add(BatchNormalization())
model3.add(Dropout(0.5))

model3.add(Flatten())
model3.add(Dense(64, activation='relu'))
model3.add(Dropout(0.5))

model3.add(Dense(1, activation='sigmoid'))

In [27]:
x_train=x_train_b
y_train=y_train_b

In [28]:
model3.compile(optimizer=Adam(learning_rate=0.0001), loss = 'binary_crossentropy', metrics=['accuracy'])

In [29]:
history = model3.fit(x_train, y_train, epochs=20, verbose=1)

Epoch 1/20
 1/20 [>.............................] - ETA: 9s - loss: 0.8763 - accuracy: 0.5938

2022-07-27 20:48:16.948011: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


20/20 [==============================] - 1s 32ms/step - loss: 1.0202 - accuracy: 0.5441
Epoch 2/20
20/20 [==============================] - 0s 15ms/step - loss: 0.9007 - accuracy: 0.6193
Epoch 3/20
20/20 [==============================] - 0s 15ms/step - loss: 0.8101 - accuracy: 0.6667
Epoch 4/20
20/20 [==============================] - 0s 15ms/step - loss: 0.7171 - accuracy: 0.6634
Epoch 5/20
20/20 [==============================] - 0s 16ms/step - loss: 0.7266 - accuracy: 0.6961
Epoch 6/20
20/20 [==============================] - 0s 15ms/step - loss: 0.6086 - accuracy: 0.7337
Epoch 7/20
20/20 [==============================] - 0s 15ms/step - loss: 0.6470 - accuracy: 0.7451
Epoch 8/20
20/20 [==============================] - 0s 16ms/step - loss: 0.6110 - accuracy: 0.7794
Epoch 9/20
20/20 [==============================] - 0s 16ms/step - loss: 0.5588 - accuracy: 0.7859
Epoch 10/20
20/20 [==============================] - 0s 16ms/step - loss: 0.5339 - accuracy: 0.7745
Epoch 11/20
20/20 [=

In [30]:
print(scores(model3))

([0.9377162629757786, 0.7222222222222222, 0.1059062850593256, 0.8611111111111112, 0.09477124183006536, 0.1830065359477124], [0.9003619331488184, 0.7027890142644241, 0.12300128296247782, 0.855072463768116, 0.11688311688311688, 0.18032786885245902])


2022-07-27 20:48:26.496900: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
/Users/weixiao/miniforge3/envs/tensorflow/lib/python3.9/site-packages/tensorflow/python/keras/engine/sequential.py:430: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


In [46]:
model4 = Sequential()
model4.add(LSTM(32, input_shape=(1,13), activation='relu', return_sequences=True))
model4.add(Dropout(0.2))

model4.add(LSTM(64, activation='relu'))
model4.add(Dropout(0.5))

model4.add(Dense(64, activation='relu'))
model4.add(Dropout(0.5))

model4.add(Flatten())
model4.add(Dense(1, activation='sigmoid'))

In [47]:
x_train_b = x_train_b.reshape(x_train_b.shape[0],1,x_train_b.shape[1])

In [48]:
x_train=x_train_b
y_train=y_train_b

In [49]:
model4.compile(optimizer=Adam(learning_rate=0.001), loss = 'binary_crossentropy', metrics=['accuracy'])

In [50]:
history=model4.fit(x_train_b, y_train_b, epochs=20)

Epoch 1/20


2022-07-27 20:52:47.524127: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


20/20 [==============================] - 2s 26ms/step - loss: 0.6919 - accuracy: 0.5408
Epoch 2/20
20/20 [==============================] - 0s 20ms/step - loss: 0.6831 - accuracy: 0.7092
Epoch 3/20
20/20 [==============================] - 0s 21ms/step - loss: 0.6602 - accuracy: 0.7974
Epoch 4/20
20/20 [==============================] - 0s 20ms/step - loss: 0.5915 - accuracy: 0.8301
Epoch 5/20
20/20 [==============================] - 0s 20ms/step - loss: 0.4703 - accuracy: 0.8431
Epoch 6/20
20/20 [==============================] - 0s 20ms/step - loss: 0.3716 - accuracy: 0.8611
Epoch 7/20
20/20 [==============================] - 0s 20ms/step - loss: 0.3449 - accuracy: 0.8709
Epoch 8/20
20/20 [==============================] - 0s 21ms/step - loss: 0.3281 - accuracy: 0.8693
Epoch 9/20
20/20 [==============================] - 0s 22ms/step - loss: 0.3266 - accuracy: 0.8742
Epoch 10/20
20/20 [==============================] - 0s 21ms/step - loss: 0.3179 - accuracy: 0.8791
Epoch 11/20
20/20 [=

In [51]:
print(scores(model4))

2022-07-27 20:52:59.116120: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
/Users/weixiao/miniforge3/envs/tensorflow/lib/python3.9/site-packages/tensorflow/python/keras/engine/sequential.py:430: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


([0.9616280063223547, 0.7941176470588235, 0.07538085723677689, 0.8970588235294118, 0.1437908496732026, 0.06209150326797386], [0.9156908665105385, 0.7294017457951885, 0.11707587287667359, 0.8623188405797102, 0.15584415584415584, 0.11475409836065574])
